In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import numpy as np

In [20]:
df = pd.read_csv('../results/clip_analysis.csv')

In [3]:
df['condition'] = np.where(df['word'].str.lower() == df['color'].str.lower(), 'congruent', 'incongruent')
df.loc[df['word'] == 'XXXX', 'condition'] = 'neutral'
df



,word,color,class_set,class_name,probability,condition
0,BLUE,red,word_only,the word is blue,0.985489,incongruent
1,BLUE,red,word_only,the word is red,0.014511,incongruent
2,BLUE,red,color_only,the color is blue,0.968680,incongruent
3,BLUE,red,color_only,the color is red,0.031320,incongruent
4,BLUE,red,combined,the word is blue in red color,0.766408,incongruent
5,BLUE,red,combined,the word is blue in blue color,0.129493,incongruent
6,BLUE,red,combined,the word is red in blue color,0.100890,incongruent
7,BLUE,red,combined,the word is red in red color,0.003209,incongruent
8,BLUE,blue,word_only,the word is blue,0.998753,congruent
9,BLUE,blue,word_only,the word is red,0.001247,congruent


In [38]:
class_names = df['class_name'].unique()
fig = px.bar(df, 
             y='probability', 
             x='class_name', 
             facet_col='class_set', 
             color='color', 
             color_discrete_map={'blue': 'blue', 'red': 'red'},
             facet_row='word',
             hover_data=df.columns.tolist(),
             barmode='group', category_orders={'class_name': class_names})
# the the x-axis does not need to be the same
fig.update_xaxes(matches=None) 
for col in range(1, 4):
    for row in range(2, 4):
        fig.update_xaxes(matches=f'x{col}', row=row, col=col)  # x1 matches x2
fig.update_layout(height=800, width=1200, title_text='CLIP Stroop Test')
fig.show()


In [33]:
df.query("(word == 'RED') & class_set == 'combined'")

,word,color,class_set,class_name,probability
20,RED,red,combined,the word is red in red color,0.645096
21,RED,red,combined,the word is red in blue color,0.253324
22,RED,red,combined,the word is blue in red color,0.101351
23,RED,red,combined,the word is blue in blue color,0.000229
28,RED,blue,combined,the word is red in blue color,0.632105
29,RED,blue,combined,the word is blue in red color,0.196627
30,RED,blue,combined,the word is red in red color,0.169246
31,RED,blue,combined,the word is blue in blue color,0.002023


In [24]:
fig = px.bar(df.query("(word == 'BLUE') & class_set == 'word_only'"), 
             y='probability', 
             x='class_name', 
             facet_col='class_set', 
             color='color', 
             color_discrete_map={'blue': 'blue', 'red': 'red'},
             facet_row='word',
             hover_data=df.columns.tolist(),
             barmode='group')
fig.show()
